In [1]:
# 验证gcn特征来源

In [1]:
# 作者：常晓松
# 作用：日志等级预测
# 时间： 2024/7/9 19:31
from imblearn.over_sampling import SMOTE
from scipy.sparse import diags
import scipy
import stellargraph as sg
from playsound import playsound
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import *
from keras import Model, Input
from keras import losses
from keras.callbacks import EarlyStopping
import torch
import os
import matplotlib.pyplot as plt
import datetime
import sklearn as sk
from sklearn.metrics import balanced_accuracy_score

from random import random
from stellargraph.layer.gcn import GraphConvolution, GatherIndices
from keras.layers import Lambda

from scipy.spatial.distance import pdist
import pandas as pd
from tqdm import tqdm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import pymysql
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
import pickle
from sklearn.model_selection import train_test_split  # 出留法
from keras.layers import Convolution1D, MaxPooling1D, LSTM
import random
import numpy
import math
from statistics import mean

def get_ave_and_standard_error(data_list):

    SD_P = numpy.std(data_list, ddof=0)
    n = len(data_list)

    ave = mean(data_list)
    standard_error = SD_P / math.sqrt(n)
    return round(ave,3),round(standard_error,3)

def get_connection():
    global conn
    host = "127.0.0.1"
    user = "root"
    password = 'chang123'
    db = 'predict_log_final'
    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        charset='utf8',
        # autocommit=True,    # 如果插入数据，， 是否自动提交? 和conn.commit()功能一致。
    )
    return conn

def shuffle_data(node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable, seed):
    c = list(zip(node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable))  # 将a,b整体作为一个zip,每个元素一一对应后打乱
    random.seed(seed)
    random.shuffle(c)  # 打乱c
    node[:], semantic_vec[:],semantic_vec_one_node[:], syntatic_vec[:],leaf_method_name_vec[:],lable[:] = zip(*c)  # 将打乱的c解开
    return node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable



def get_method_name_emb(seq):
    cur = conn.cursor()
    sqli = "select method_name_emb from data_model_1 where seq=( select methodSeq from data_model_2 where seq=" + str(seq) + ")"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    emb=oneRow[0]
    cur.close()
    return eval(emb)



def get_positive_data_from_db(vec_length):
    cur = conn.cursor()
    sqli = "select vectorStruct,logNum from data_model_2 where logNum<>0 and vectorStruct is not null;"
    result = cur.execute(sqli)
    methods_padding = np.empty(shape=[0, vec_length], dtype=int)
    labels = []
    for i in range(result):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        labels.append(oneRow[1])
    return methods_padding, labels



def add_nagitive_data(methods_padding, lables):
    cur = conn.cursor()
    sqli = "select vectorStruct,logNum from data_model_2 where logNum=0 and vectorStruct is not null limit " + str(
        len(methods_padding)) + ";"
    result = cur.execute(sqli)
    for i in range(1, len(methods_padding)):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        lables.append(oneRow[1])
    return methods_padding, lables


def string_vector_to_int(methods_padding, oneRow):
    methods_padding = np.append(methods_padding, [list(map(int, oneRow[0][1:len(oneRow[0]) - 1].split(",")))], axis=0)
    return methods_padding


def float_revert_int(value_list):
    revert_value_list = []
    for i in value_list:
        revert_value_list.append(int(np.round(i)))
    value_list = revert_value_list
    return revert_value_list


def draw_confusion_matrix(y_true, y_pred, dic_lables):

    plt.cla()

    labels = []
    for key in dic_lables:
        labels.append(key)
    print_message(type(labels))
    # 小数转整数
    y_pred_int = float_revert_int(y_pred)
    sns.set()
    f, ax = plt.subplots()
    C2 = confusion_matrix(y_true,y_pred,  labels=labels)
    print_message(C2)  # 打印出来看看
    sns.heatmap(C2, annot=True, fmt='.20g', ax=ax, cmap="YlGnBu")  # 画热力图
    ax.set_title('confusion matrix')  # 标题
    ax.set_xlabel('true')  # x轴
    ax.set_ylabel('predict')  # y轴
    save_pic(plt, '混淆矩阵')
    # plt.show()
    return plt


def save_pic(plt, file_name):
    # 创建目录
    dirs = 'C:\\Users\\chang\\Desktop\\日志工作空间\\实验图片\\'
    t = time.strftime('%Y-%m-%d-%H', time.localtime(int(time.time())))
    dirs = dirs + t
    file = dirs + '\\' + file_name + '.png'
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    if os.path.exists(file):
        os.remove(file)
    # 保存图片
    plt.savefig(file)  # 保存图片


def plot_value(y_true, y_pred):
    balance_accuracy = balanced_accuracy_score(y_true, y_pred)
    precision = sk.metrics.precision_score(y_true, y_pred)
    recall = sk.metrics.recall_score(y_true, y_pred)
    f1_value = sk.metrics.f1_score(y_true, y_pred)
    return balance_accuracy, precision, recall, f1_value


def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


def plot_graphs(history_loc, type_loc):
    plt.cla()

    plt.plot(history_loc.history[type_loc])
    plt.plot(history_loc.history['val_' + type_loc])
    plt.xlabel('Epochs')
    plt.ylabel(type_loc)
    plt.legend([type_loc, 'val_' + type_loc])
    save_pic(plt, type_loc)
    # plt.show()
    return plt


def str_revert_to_list(methods_padding, max_len):
    methods_padding_list = np.empty(shape=[0, max_len], dtype=int)

    for methods_padding_one in methods_padding:
        list1 = methods_padding_one.split(',')
        list1 = list(map(int, list1))
        methods_padding_list = np.append(methods_padding_list, [np.array(list1)], axis=0)  # 添加整行元素，axis=1添加整列元素
    return methods_padding_list


def get_object(save_absolute_path):
    summer_load = None
    with open(save_absolute_path, 'rb') as f:
        summer_load = pickle.load(f)  # read file and build object
    return summer_load


def save_object(summer_save, save_absolute_path):
    summer_save = pickle.dumps(summer_save)
    with open(save_absolute_path, 'wb') as f:  # open file with write-mode
        f.write(summer_save)  # serialize and save object


def get_parent(vec_chair):
    cur = conn.cursor()
    sqli = "select parentId from data_model_2 where seq=" + str(vec_chair[len(vec_chair) - 1]) + ";"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    if oneRow[0] == 0:
        return vec_chair
    else:
        vec_chair.append(oneRow[0])
        return get_parent(vec_chair)


def get_chair_vec(vec_chair, i,maxlen):
    cur = conn.cursor()
    one_leaf_list = []
    leaf = True
    message_type = i
    for one_leaf in vec_chair:
        sqli = "select a.vectorSemantic,REPLACE(REPLACE(vectorStruct,'[',''),']',''),logNum ,REPLACE(REPLACE(syntacticMessage,'[',''),']','') " \
               "from data_model_2 a  where seq=" + str(one_leaf) + ";"
        result = cur.execute(sqli)
        oneRow = cur.fetchone()
        vectorSemantic = oneRow[0]
        vectorStruct = oneRow[1]
        logNum = oneRow[2]
        syntacticMessage = oneRow[3]
        if vectorSemantic != "[]":
            if message_type == 1 or message_type == 3:
                if vectorSemantic is not None :
                    if len(vectorSemantic)==0:
                        vectorSemantic='0'
                    list_semantic = vectorSemantic.split(',')
                    list_semantic = list(map(float, list_semantic))
                    list_semantic_tmp = pad_sequences([list_semantic], maxlen=maxlen,
                                                      padding="post", truncating="post", dtype='float32')
                    list_semantic = list_semantic_tmp[0]
                    one_leaf_list.extend(list_semantic)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 2:
                if syntacticMessage is not None :
                    if len(syntacticMessage)==0:
                        syntacticMessage='0'
                    list_syntactic = syntacticMessage.split(',')
                    list_syntactic = list(map(float, list_syntactic))
                    list_syntactic_tmp = pad_sequences([list_syntactic], maxlen=maxlen,
                                                       padding="post", truncating="post", dtype='float32')
                    list_syntactic = list_syntactic_tmp[0]
                    one_leaf_list.extend(list_syntactic)
    cur.close()
    return one_leaf_list


def print_message(message):
    t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(time.time())))
    print('时间：' + t, end=" ", flush=True)
    print(message, flush=True)



def repetition_list(a):
    repetition = 6
    c = []
    for a_one in a:
        b = []
        for a_single in a_one:
            for i in range(repetition):
                b.append(a_single)
        c.append(b)
    return c


def overlap_channel(a, b):
    element_num = 0
    height = 0
    width = 0
    if len(a.shape) == 2:
        element_num = a.shape[0]
        height = 1
        width = a.shape[1]
    elif len(a.shape) == 3:
        element_num = a.shape[0]
        height = a.shape[1]
        width = a.shape[2]
    else:
        raise Exception(print_message('通道叠加 不支持这类维度:' + str(a.shape)))

    a = a.reshape(element_num, 1, height, width)
    b = b.reshape(element_num, 1, height, width)
    a = a.transpose(1, 0, 2, 3)
    b = b.transpose(1, 0, 2, 3)
    c = np.vstack((a, b))
    c = c.transpose(1, 2, 3, 0)
    return c


def check_list_exist(A, B):
    exist = False
    for row in A:
        if (row == B).all():
            exist = True
    return exist


def get_convert_data(A, B):
    C = []
    for A_one in A:
        for B_one in B:
            if (np.array(A_one) == np.array(B_one)).all():
                C.append(A_one)
    return C


def cosDist(A, B):
    return pdist(np.vstack([A, B]), 'cosine')  # np.sqrt(sum(np.power((A - B), 2)))


def get_normal_data(data_point, data_lable):
    neigh_num_a = 5
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
    return data_point_clear, data_lable_clear, abnormal_old


##仅支持0 - 1 分类
def change_abnormal(data_point, data_lable):
    neigh_num_a = 10
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
        else:
            ##修改noise标签
            data_point_clear.append(one_point)
            if one_lable == 0:
                data_lable_clear.append(1)
            else:
                data_lable_clear.append(0)
    return data_point_clear, data_lable_clear, abnormal_old


def clear_data(train_vec_tmp, train_labels, zero_index):
    group_num = 300
    train_vec_tmp_clear = []
    train_labels_clear = []
    abnormal_num = 0
    for b in range(int(len(train_vec_tmp) / group_num) + 1):
        begin = b * group_num + zero_index
        end = (b + 1) * group_num
        if end > len(train_vec_tmp):
            end = len(train_vec_tmp)
        train_vec_group_one, train_labels_group_one, abnormal_old = change_abnormal(train_vec_tmp[begin:end],
                                                                                    train_labels[begin:end])
        train_vec_tmp_clear.extend(train_vec_group_one)
        train_labels_clear.extend(train_labels_group_one)
        abnormal_num = abnormal_num + len(abnormal_old)
    print_message('noise数据量：' + str(abnormal_num))
    print_message('训练总数据量：' + str(len(train_vec_tmp)))
    train_vec_tmp = train_vec_tmp_clear
    train_labels = train_labels_clear
    return train_vec_tmp, train_labels


def play_title_message():
    playsound('D:/python/log_predict/file/结束提示.mp3')


def get_tree_model_together(struct_vec_length_one,semantic_vec_length_one,syntatic_vec_length_one
                            ,num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        keras.layers.InputLayer(struct_vec_length_one)
    ])
    model2 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(semantic_vec_length_one, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(syntatic_vec_length_one, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model_together = keras.layers.concatenate([model3.output, model2.output])
    x = Dropout(0.4)(model_together)
    x = Dense(24, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    final_output = Dense(24, activation='relu')(x)
    model_together = Model(inputs=[model3.input, model2.input], outputs=final_output)
    return model_together



def seperate_data(struct_leaf_vec, semantic_leaf_vec, syntatic_leaf_vec,
                  lable, leaf,seed):
    struct_train_vec, struct_test_vec, \
    semantic_train_vec, semantic_test_vec, \
    syntatic_train_vec, syntatic_test_vec, \
    struct_train_labels, struct_test_labels, \
    struct_train_leaf, struct_test_leaf = train_test_split(
        struct_leaf_vec, semantic_leaf_vec, syntatic_leaf_vec,
        lable, leaf,
        test_size=0.2,
        random_state=seed)
    struct_verify_vec, struct_test_vec, \
    semantic_verify_vec, semantic_test_vec, \
    syntatic_verify_vec, syntatic_test_vec, \
    struct_verify_labels, struct_test_labels, \
    struct_verify_leaf, struct_test_leaf = train_test_split(
        struct_test_vec, semantic_test_vec, syntatic_test_vec,
        struct_test_labels,
        struct_test_leaf,
        test_size=0.5,
        random_state=seed)
    return struct_train_vec, struct_test_vec, \
           semantic_train_vec, semantic_test_vec, \
           syntatic_train_vec, syntatic_test_vec, \
           struct_train_labels, struct_test_labels, \
           struct_verify_vec, semantic_verify_vec, \
           syntatic_verify_vec, struct_verify_labels, struct_verify_leaf

def get_lable(conn, seq):
    seq_db = 0
    leaf = ''
    lable = 0
    sqli = "select seq,leaf,CASE WHEN logNum > 0 THEN  1  ELSE 0 END from data_model_2 where seq=" + seq
    cur = conn.cursor()
    result = cur.execute(sqli)
    for i in range(result):
        oneRow = cur.fetchone()
        seq_db = oneRow[0]
        leaf = oneRow[1]
        lable = oneRow[2]
    cur.close()
    return seq_db, leaf, lable



def get_feature_from_file(file_name, node_all, node_vec):
    with open(file_name, 'r') as file:
        line = file.readline()
        while line:
            one_row = line.split(' ')
            if len(one_row) > 2:
                node_all.append(one_row[0])
                one_row[len(one_row) - 1] = one_row[len(one_row) - 1].replace('\n', '')
                one_row = list(map(float, one_row))
                node_vec.append(list(map(float, one_row[1:])))
            line = file.readline()
    return node_all, node_vec


def convert_metric_to_list(project_name, feature_type, percentage, metric_type, tool_type):
    one_row = []
    one_row.append(project_name)
    one_row.append(feature_type)
    one_row.append(percentage)
    one_row.append(metric_type)
    one_row.append(tool_type)
    return one_row




def get_call_graph(graph_leng, num_word, embedding_dim):
    model3 = tf.keras.models.Sequential([
        Convolution1D(input_shape=(graph_leng, 1), filters=32, kernel_size=5, strides=1, padding='same',
                      activation='relu', name='call-message'),
        MaxPooling1D(pool_size=2, strides=2, padding='same', ),
        Convolution1D(64, 5, strides=1, padding='same', activation='relu'),
        MaxPooling1D(2, 2, 'same'),
        Flatten(),
        Dense(69, activation='relu'),
        Dense(2, activation='softmax'),
    ])
    return model3

def get_method_name_model(method_name_x, method_name_y, num_word, embedding_dim):

    model1 = tf.keras.models.Sequential([
        keras.layers.Masking(mask_value=0, input_shape=(method_name_x, method_name_y)),
        LSTM(units=8,activation='sigmoid'),
        #         LSTM(units=32, activation='tanh'),
        #         Flatten(),
        Dense(53, activation='relu'),
        Dense(2, activation='softmax'),
    ])
    return model1

def get_tree_model_together(semantic_vec_length_chain,syntatic_vec_length_chain,num_word,embedding_dim):
    model2 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(semantic_vec_length_chain, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(syntatic_vec_length_chain, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        #         LSTM(units=32, activation='tanh', return_sequences=True),
        #         Flatten(),
    ])
    model_together = keras.layers.concatenate([model3.output, model2.output])
    x = Dropout(0.2)(model_together)
    final_output = Dense(43, activation='relu')(x)
    model_together = Model(inputs=[model3.input, model2.input], outputs=final_output)
    return model_together


def get_method_name_call_graph_model(method_dem_x,method_dem_y,call_graph_len,num_word,embedding_dim):
    method_name_model = get_method_name_model(method_dem_x,method_dem_y,  num_word, embedding_dim)
    call_graph_model = get_call_graph(call_graph_len,  num_word, embedding_dim)
    m_j_merged_a = keras.layers.concatenate([method_name_model.output,call_graph_model.output],axis=-1)
    dense1_a = Dense(4,activation='tanh')(m_j_merged_a)
    final_output = Dropout(0.2)(dense1_a)
    model_together_final = Model(inputs=[ method_name_model.input,call_graph_model.input],outputs=final_output)
    return model_together_final


def get_featrue_and_label(project_name):
    # 1-语法 2-语义 3-结合(本树+调用关系)
    # 4-语法（新）5-纯调用关系 6-结合(本树)
    select_model = 6
    semantic_vec_length = 50
    tree_deep=4
    semantic_vec_length_chain =  semantic_vec_length* tree_deep
    syntatic_vec_length_chain = 30 * tree_deep
    bro_vec_length_semantic = 0  # 304
    # 获取每个叶子节点
    print_message("获取节点")
    node = []
    lable = []
    semantic_leaf_one_node = []
    semantic_leaf_vec = []
    syntatic_leaf_vec = []
    theme_leaf_vec = []

    conn = get_connection()
    cur = conn.cursor()

    sqli = "select seq,(select method_name_emb from data_model_1 where seq=a.methodSeq)," \
           "CASE WHEN logNum > 0 THEN  1  ELSE 0 END " \
           "from  data_model_2 a where  methodSeq in " \
           "(select seq from data_model_1 where projectName='"+project_name+"') "
    result = cur.execute(sqli)
    # and methodSeq<59537
    for i in range(result):
        oneRow = cur.fetchone()
        node.append(oneRow[0])
        theme_leaf_vec.append(eval(oneRow[1]))
        lable.append(oneRow[2])
    print_message("获取节点特征")
    for one_leaf in node:
        vec_chain = []
        for i in range(1, 4):
            if i == 1:
                maxlen = semantic_vec_length_chain
                vec_chain = []
                vec_chain.append(one_leaf)
                vec_chain = get_parent(vec_chain)
            elif i==2:
                maxlen = syntatic_vec_length_chain
                vec_chain = []
                vec_chain.append(one_leaf)
                vec_chain = get_parent(vec_chain)
            elif i==3:
                maxlen=semantic_vec_length
                vec_chain = []
                vec_chain.append(one_leaf)

            # 根据list中的记录查询节点记录，组合向量
            one_leaf_vec = get_chair_vec(vec_chain, i,maxlen//tree_deep)
            # 划分为等长向量
            one_leaf_vec_padding_tmp = pad_sequences([one_leaf_vec], maxlen=maxlen,
                                                     padding="post", truncating="post", dtype='float32')
            one_leaf_vec_padding_tmp = one_leaf_vec_padding_tmp[0]

            if i == 1:
                semantic_leaf_vec.append(one_leaf_vec_padding_tmp)
            elif i==2:
                syntatic_leaf_vec.append(one_leaf_vec_padding_tmp)
            elif i==3:
                semantic_leaf_one_node.append(one_leaf_vec_padding_tmp)

    cur.close()
    conn.close()
    print_message("完成")
    return node,semantic_leaf_vec,semantic_leaf_one_node,syntatic_leaf_vec,theme_leaf_vec,lable


def get_feature_from_file(file_name, node_all, node_vec):
    with open(file_name, 'r') as file:
        line = file.readline()
        while line:
            one_row = line.split(' ')
            if len(one_row) > 2:
                node_all.append(one_row[0])
                one_row[len(one_row) - 1] = one_row[len(one_row) - 1].replace('\n', '')
                one_row = list(map(float, one_row))
                node_vec.append(list(map(float, one_row[1:])))
            line = file.readline()
    return node_all, node_vec



def build_data(node,semantic_vec,edge,lable):
    features_matrix_tmp=[]
    for one_node,feature_value in zip(node,semantic_vec):
        features_matrix_tmp.append(feature_value)

    features_matrix=np.array(features_matrix_tmp)
    a = {}
    index=0
    for i in features_matrix:
        a[index]=[int(i) for i in i.tolist()]
        index+=1


    max_feature = np.max([v for v_list in a.values() for v in v_list])
    features_matrix = np.zeros(shape = (len(list(a.keys())), max_feature+1))

    i = 0
    for k, vs in tqdm(a.items()):
        for v in vs:
            features_matrix[i, v] = 1
        i+=1



    class data:
        x=[]
        y=[]
        edge_index=[]
        train_mask=[]
        val_mask=[]
        test_mask=[]

    data=data()
    node=node
    label=lable
    edge=edge
    feature=[]

    feature = features_matrix
    # for i in node:
    #     feature.append(seq_feature[i])

    train_rate=0.6
    val_rate=0.2
    test_rate=0.2
    number=0
    node_new={}
    for i in node:
        node_new[i]=number
        number=number+1
    data.x=torch.Tensor(feature)
    edge_new=[[],[]]
    for i in edge:
        edge_new[0].append(node_new[i[0]])
        edge_new[1].append(node_new[i[1]])

    data.edge_index=torch.LongTensor(edge_new)

    data.y=torch.LongTensor(label)
    train_num=int(len(node)*train_rate)
    train_mask=[ False for i in node]
    while train_num>0:
        index=0
        for i in node:
            if train_num>0 and random.random()>=0.5 and not train_mask[index]:
                train_mask[index]=True
                train_num=train_num-1
            index=index+1
    data.train_mask=train_mask
    test_num=int(len(node)*test_rate)

    test_mask=[ False for i in node]
    while test_num>0:
        index=0
        for i,j in zip(test_mask,train_mask):
            if test_num>0 and not i and not j:
                if random.random()>=0.5:
                    test_mask[index]=True
                    test_num=test_num-1
            index=index+1

    data.test_mask=test_mask
    val_num=int(len(node)*val_rate)

    val_mask=[ False for i in node]
    while val_num>0:
        index=0
        for i,j,z in zip(val_mask,train_mask,test_mask):
            if val_num>0 and not i and not j and not z:
                if random.random()>=0.5:
                    val_mask[index]=True
                    val_num=val_num-1
            index=index+1
    data.val_mask=val_mask

    return data

def get_node_indices(G, ids):
    # find the indices of the nodes
    node_ids = np.asarray(ids)
    flat_node_ids = node_ids.reshape(-1)

    flat_node_indices = G.node_ids_to_ilocs(flat_node_ids) # in-built function makes it really easy
    # back to the original shape
    node_indices = flat_node_indices.reshape(1, len(node_ids)) # add 1 extra dimension

    return node_indices

def splite_data(vec_x_sem,vec_x_syn,vec_x_graph,vec_theme,struc_vec,
                vec_y,mask):
    vec_sem=[]
    vec_syn=[]
    vec_graph=[]
    vec_method_theme=[]
    vec_struc=[]
    lable=[]
    for i,j,h,w,a,g,z  in zip(vec_x_sem,vec_x_syn,
                              vec_x_graph,vec_theme,
                              struc_vec,vec_y,
                              mask):
        if z:
            vec_sem.append(i)
            vec_syn.append(j)
            vec_graph.append(h)
            vec_method_theme.append(w)
            vec_struc.append(a)
            lable.append(g)
    return vec_sem,vec_syn,vec_graph,vec_method_theme,vec_struc,lable
def get_method_name_emb(seq):
    cur = conn.cursor()
    sqli = "select method_name_emb from data_model_1 where seq=( select methodSeq from data_model_2 where seq=" + str(seq) + ")"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    emb=oneRow[0]
    cur.close()
    return eval(emb)


# 得到多分枝模型
def get_graph_model_4(semantic_vec,graph_leng,syntactic_len,method_name_x, method_name_y,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model2 = tf.keras.models.Sequential([
        Input(shape=(graph_leng,1)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        Flatten(),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model4 = tf.keras.models.Sequential([
        keras.layers.Masking(mask_value=0, input_shape=(method_name_x, method_name_y)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    x = keras.layers.concatenate([model1.output, model2.output, model3.output,model4.output])
#     model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)
#     x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
#     x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model1.input, model2.input, model3.input,model4.input], outputs=final_output)
    return model_together

def get_graph_model_3(semantic_vec,graph_leng,syntactic_len,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model2 = tf.keras.models.Sequential([
        Input(shape=(graph_leng,1)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        Flatten(),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    x = keras.layers.concatenate([model1.output, model2.output, model3.output])

#     model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)
#     x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
#     x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model1.input, model2.input, model3.input], outputs=final_output)
    return model_together

def get_graph_model_2(semantic_vec,syntactic_len,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
    ])
    x = keras.layers.concatenate([model1.output, model3.output])
#     model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)

#     x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
#     x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model1.input,model3.input], outputs=final_output)
    return model_together
def get_graph_model_1(semantic_vec,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        (Dropout(0.2)),
        Dense(2, activation='softmax'),
    ])
    return model1
def get_graph_model_gcn(graph_vec_len,
                        num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(graph_vec_len,1)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        (Dropout(0.2)),
        Dense(2, activation='softmax'),

    ])
    return model1
def save_performance(content,ac,pr,re,f1):
    f = open("print-message.txt", mode='a')
    f.write(str(datetime.datetime.now())+'\t'+ content+' \t'+str(ac)+
            '\t'+str(pr)+ '\t'+str(re)+'\t'+ str(f1)+'\n')
    f.close()

In [3]:

num_epoch = 100  # 训练周期
feature_length = "10"
one_project = 'zeppelin-master'
node_all = []
node_vec = []

node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable = get_featrue_and_label(one_project)


semantic_vec_one_node_tmp=[]
for one in semantic_vec_one_node:
    tmp=[ int(i) for i in one if i!=0]
    semantic_vec_one_node_tmp.append(tmp)
semantic_vec_one_node=semantic_vec_one_node_tmp

randomValue=int(random.random()*100)
node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable=shuffle_data(node, semantic_vec,semantic_vec_one_node, syntatic_vec,leaf_method_name_vec,lable,randomValue)


时间：2024-08-15 19:03:32 获取节点
时间：2024-08-15 19:28:25 获取节点特征
时间：2024-08-15 19:33:30 完成


In [4]:
file_name = "D:\\研究生\\研究内容\\研究生开题\\" \
            "工作空间\\最终版实验程序和数据备份\\" \
            "dataset\\struc2vec-vec\\barbell-" + feature_length + "-" + one_project + ".emb"
node_all, node_vec = get_feature_from_file(file_name, node_all, node_vec)
tmp={}
for one ,vec in zip(node_all,node_vec):
    tmp[int(one)]=vec
struc_vec=[]
struc_vec_dict={}
for i,j in zip(node_all,node_vec):
    struc_vec_dict[int(i)]=j
for i in node:
    struc_vec.append(struc_vec_dict[i])

In [5]:
semantic_vec_tmp=np.array(semantic_vec).astype(int)
syntatic_vec_tmp=np.array(syntatic_vec).astype(int)

In [6]:
feature_all=[]
for i, j in zip(semantic_vec_tmp,syntatic_vec_tmp):
    tmp=[]
    tmp.extend(i)
    tmp.extend(j)
    feature_all.append(tmp)

In [7]:
# 得到图
number=0
node_new={}
for i in node:
    node_new[i]=number
    number=number+1

edge=[]#[[28,16],[16,85],[85,46],[28,16],[16,85],[85,46],[16,85],[85,46]]

conn = get_connection()

cur = conn.cursor()
sqli = "select parentId,seq " \
       "from data_model_2 " \
       "where methodSeq in " \
       "(select seq from data_model_1 where projectName='"+one_project+"') " \
       "and parentId<>0   " \
       "union " \
       "select callBlockSeq,calledBlockSeq " \
       "from call_graph_data " \
       "where callMethodSeq in " \
    "(select seq from data_model_1 where projectName='"+one_project+"')   "
result = cur.execute(sqli)
#
#and callMethodSeq<59537 and calledMethodSeq<59537
for i in range(result):
    oneRow = cur.fetchone()
    edge_tmp=[]
    edge_tmp.append(node_new[oneRow[0]])
    edge_tmp.append(node_new[oneRow[1]])
    edge.append(edge_tmp)
cur.close()
conn.close()
#构建图 单语义节点
features = {}
index=0
index_all=[]
for feature_one in feature_all:
    features[index] = feature_one
    index_all.append(index)
    index+=1
max_feature = np.max([v for v_list in features.values() for v in v_list])
features_matrix = np.zeros(shape = (len(list(features.keys())), max_feature+1))

i = 0
for k, vs in tqdm(features.items()):
    for v in vs:
        features_matrix[i, v] = 1
    i+=1

node_features = pd.DataFrame(features_matrix, index = features.keys()) # into dataframe for StellarGraph

edge = pd.DataFrame(edge,columns=['source', 'target']) # 将第一维度数据转为为行，第二维度数据转化为列，即 3 行 2 列，并设置列标签
edge.columns = ['source', 'target']


G = sg.StellarGraph(node_features, edge)
print(G.info())
targets = pd.DataFrame(data=None,columns=['id','name','ml_target']) # into dataframe for StellarGraph
import pandas as pd

index=0
for key,label_tmp in zip(features.keys(),lable):
    targets.loc[index] = [key,'-',label_tmp] 
    index=index+1
# targets.columns = ['name', 'ml_target']
targets.index = targets.id
targets = targets.loc[features.keys(), :]
targets.index = targets.id.astype(str)
train_pages, test_pages = train_test_split(targets, test_size=0.4,random_state=73)
val_pages, test_pages = train_test_split(test_pages, test_size=0.5,random_state=73)
print(train_pages.shape, val_pages.shape, test_pages.shape)

from sklearn.preprocessing import LabelBinarizer

target_encoding = LabelBinarizer()
train_targets = target_encoding.fit_transform(np.int64(train_pages['ml_target']))
val_targets = target_encoding.transform(np.int64(val_pages['ml_target']))
test_targets = target_encoding.transform(np.int64(test_pages['ml_target']))

# 得到gcn特征

# Get the adjacency matrix
A = G.to_adjacency_matrix(weighted=False)
# Add self-connections
A_t = A + scipy.sparse.diags(np.ones(A.shape[0]) - A.diagonal())
# Degree matrix to the power of -1/2
D_t = scipy.sparse.diags(np.power(np.array(A.sum(1)), -0.5).flatten(), 0)
# Normalise the Adjacency matrix
A_norm = A.dot(D_t).transpose().dot(D_t).todense()
A_input = np.expand_dims(A_norm, 0)


# Define the function to get these indices
def get_node_indices(G, ids):
    # find the indices of the nodes
    node_ids = np.asarray(ids)
    flat_node_ids = node_ids.reshape(-1)

    flat_node_indices = G.node_ids_to_ilocs(flat_node_ids) # in-built function makes it really easy
    # back to the original shape
    node_indices = flat_node_indices.reshape(1, len(node_ids)) # add 1 extra dimension
    
    return node_indices

# Get indices
train_indices = get_node_indices(G, np.int64(train_pages.index))
val_indices = get_node_indices(G, np.int64(val_pages.index))
test_indices = get_node_indices(G, np.int64(test_pages.index))

# Expand dimensions
features_input = np.expand_dims(features_matrix, 0)
A_input = np.expand_dims(A_norm, 0)

y_train = np.expand_dims(train_targets, 0)
y_val = np.expand_dims(val_targets, 0)
y_test = np.expand_dims(test_targets, 0)


100%|██████████████████████████████████████████████████████████████████████████| 57818/57818 [00:06<00:00, 8938.71it/s]


StellarGraph: Undirected multigraph
 Nodes: 57818, Edges: 65082

 Node types:
  default: [57818]
    Features: float32 vector, length 6948
    Edge types: default-default->default

 Edge types:
    default-default->default: [65082]
        Weights: all 1 (default)
        Features: none
(34690, 3) (11564, 3) (11564, 3)


# 旧工作 deeplog

In [8]:

# 数据集切分
print('数据集切分正常')
all_indices = get_node_indices(G, np.int64(targets.index))

vec_x_graph=[]
for i in semantic_vec:
    vec_x_graph.append(0)
train_mask=[]
val_mask=[]
test_mask=[]
for i in all_indices[0]:
    if i in train_indices[0]:
        train_mask.append(True)
    else:
        train_mask.append(False)
    if i in val_indices[0]:
        val_mask.append(True)
    else:
        val_mask.append(False)
    if i in test_indices[0]:
        test_mask.append(True)
    else:
        test_mask.append(False)
train_vec_sem,train_vec_syn, \
train_vec_graph,train_vec_theme,train_vec_struct,train_lable=splite_data \
    (semantic_vec,syntatic_vec,vec_x_graph,leaf_method_name_vec,struc_vec,
     lable,train_mask)
val_vec_sem,val_vec_syn, \
val_vec_graph,val_vec_theme,val_vec_struct,val_lable=splite_data \
    (semantic_vec,syntatic_vec,vec_x_graph,leaf_method_name_vec,struc_vec,
     lable,val_mask)
test_vec_sem,test_vec_syn, \
test_vec_graph,test_vec_theme,test_vec_struct,test_lable=splite_data \
    (semantic_vec,syntatic_vec,vec_x_graph,leaf_method_name_vec,struc_vec,
     lable,test_mask)


数据集切分正常


In [9]:
num_tmp=0
for i in train_mask:
    if i:
        num_tmp+=1
print(num_tmp/len(train_mask))
num_tmp=0
for i in train_lable:
    if i == 1:
        num_tmp+=1
print(num_tmp/len(train_lable))
train_lable_one_hot=tf.keras.utils.to_categorical(train_lable, num_classes=2)
val_lable_one_hot=tf.keras.utils.to_categorical(val_lable, num_classes=2)
test_lable_one_hot=tf.keras.utils.to_categorical(test_lable, num_classes=2)
# y_train.tolist()

0.5999861634785015
0.1123378495243586


# DeepLog1.0

In [10]:
def get_tree_model_together(semantic_len,syntatic_len,num_word, embedding_dim):
    model2 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(semantic_len, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(syntatic_len, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model_together = keras.layers.concatenate([model3.output, model2.output])
    x = Dropout(0.4)(model_together)
    x = Dense(24, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(24, activation='relu')(x)
    final_output=Dense(1, activation='sigmoid')(x)
    model_together = Model(inputs=[model3.input, model2.input], outputs=final_output)
    return model_together

model_old = get_tree_model_together(len(train_vec_sem[0]),len(train_vec_syn[0]),50000,16)

model_old.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=losses.binary_crossentropy,
    metrics=["accuracy"],
)
history = model_old.fit([np.array(train_vec_syn), 
                                    np.array(train_vec_sem)],
                 np.array(train_lable),
                 validation_data=([np.array(val_vec_syn),
                                   np.array(val_vec_sem)],
                                  np.array(val_lable)),
                 epochs=num_epoch, verbose=2)


test_predict = model_old.predict([np.array(test_vec_syn),
                                np.array(test_vec_sem)])
 
predict_label=[i[0] for i in np.array(test_predict)]
bin_pred = [1 if p > 0.5 else 0 for p in predict_label]
test_true = test_lable
correct = 0

y_pred=[]
y_true=[]
for i in range(len(bin_pred)):
    y_pred.append(bin_pred[i])
    y_true.append(test_true[i])
    if bin_pred[i] == test_true[i]:
        correct += 1
balance_accuracy, precision, recall, f1_value = plot_value(y_pred,y_true)

save_performance('DeepLog1.0', balance_accuracy, precision, recall, f1_value)



Epoch 1/100
1085/1085 - 27s - loss: 0.2161 - accuracy: 0.9073 - val_loss: 0.1767 - val_accuracy: 0.9167 - 27s/epoch - 24ms/step
Epoch 2/100
1085/1085 - 24s - loss: 0.1800 - accuracy: 0.9212 - val_loss: 0.1658 - val_accuracy: 0.9238 - 24s/epoch - 22ms/step
Epoch 3/100
1085/1085 - 24s - loss: 0.1695 - accuracy: 0.9243 - val_loss: 0.1657 - val_accuracy: 0.9278 - 24s/epoch - 22ms/step
Epoch 4/100
1085/1085 - 24s - loss: 0.1620 - accuracy: 0.9274 - val_loss: 0.1540 - val_accuracy: 0.9250 - 24s/epoch - 22ms/step
Epoch 5/100
1085/1085 - 24s - loss: 0.1561 - accuracy: 0.9290 - val_loss: 0.1553 - val_accuracy: 0.9308 - 24s/epoch - 22ms/step
Epoch 6/100
1085/1085 - 24s - loss: 0.1526 - accuracy: 0.9319 - val_loss: 0.1491 - val_accuracy: 0.9305 - 24s/epoch - 22ms/step
Epoch 7/100
1085/1085 - 24s - loss: 0.1475 - accuracy: 0.9318 - val_loss: 0.1430 - val_accuracy: 0.9306 - 24s/epoch - 22ms/step
Epoch 8/100
1085/1085 - 24s - loss: 0.1456 - accuracy: 0.9335 - val_loss: 0.1423 - val_accuracy: 0.9330 

Epoch 61/100
1085/1085 - 24s - loss: 0.1116 - accuracy: 0.9494 - val_loss: 0.1240 - val_accuracy: 0.9441 - 24s/epoch - 22ms/step
Epoch 62/100
1085/1085 - 24s - loss: 0.1090 - accuracy: 0.9500 - val_loss: 0.1267 - val_accuracy: 0.9446 - 24s/epoch - 22ms/step
Epoch 63/100
1085/1085 - 24s - loss: 0.1088 - accuracy: 0.9511 - val_loss: 0.1243 - val_accuracy: 0.9460 - 24s/epoch - 22ms/step
Epoch 64/100
1085/1085 - 24s - loss: 0.1108 - accuracy: 0.9494 - val_loss: 0.1226 - val_accuracy: 0.9457 - 24s/epoch - 22ms/step
Epoch 65/100
1085/1085 - 24s - loss: 0.1100 - accuracy: 0.9495 - val_loss: 0.1268 - val_accuracy: 0.9405 - 24s/epoch - 22ms/step
Epoch 66/100
1085/1085 - 25s - loss: 0.1078 - accuracy: 0.9503 - val_loss: 0.1264 - val_accuracy: 0.9446 - 25s/epoch - 23ms/step
Epoch 67/100
1085/1085 - 24s - loss: 0.1089 - accuracy: 0.9502 - val_loss: 0.1206 - val_accuracy: 0.9467 - 24s/epoch - 22ms/step
Epoch 68/100
1085/1085 - 24s - loss: 0.1095 - accuracy: 0.9513 - val_loss: 0.1227 - val_accuracy:

# ASE

In [ ]:
#ase 项目集合 项目内
# 作者：常晓松
# 作用：日志等级预测
# 时间： 2022/3/23 14:35
import os

import gensim
from sklearn.model_selection import StratifiedShuffleSplit

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import pymysql
from keras.layers import LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
import pickle
from keras import Model, Input
import random
import sklearn as sk

def stratified_shuffle_split(data_vec, data_lable, test_size, seed):
    train_data = []
    train_lable = []

    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    test_data = []
    test_lable = []
    for train_index, test_index in split.split(data_vec, data_lable):
        for i in train_index:
            train_data.append(data_vec[i])
            train_lable.append(data_lable[i])
        for i in test_index:
            test_data.append(data_vec[i])
            test_lable.append(data_lable[i])
    return train_data, train_lable, test_data, test_lable


def shuffle_data(leaf, lable, seed):
    import random
    c = list(zip(leaf, lable))
    random.seed(seed)
    random.shuffle(c)  # 打乱c
    leaf[:], lable[:] = zip(*c)  # 将打乱的c解开
    return leaf, lable


def get_positive_data_from_db(vec_length):
    cur = conn.cursor()
    sqli = "select vectorfused,lable_ASE from data_model_2 where lable_ASE<>0 and vectorfused is not null;"
    result = cur.execute(sqli)
    methods_padding = np.empty(shape=[0, vec_length], dtype=int)
    labels = []
    for i in range(result):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        labels.append(oneRow[1])
    return methods_padding, labels


def splite_data(methods_padding, labels):
    group_size_one = int(len(methods_padding) / 10)
    methods_train = methods_padding[0:group_size_one * 6]
    labels_train = labels[0:group_size_one * 6]
    methods_test = methods_padding[group_size_one * 6:group_size_one * 8]
    labels_test = labels[group_size_one * 6:group_size_one * 8]
    methods_verify = methods_padding[group_size_one * 8:]
    labels_verify = labels[group_size_one * 8:]
    return methods_train, labels_train, methods_test, labels_test, methods_verify, labels_verify


def add_nagitive_data(methods_padding, lables):
    cur = conn.cursor()
    sqli = "select vectorfused,lable_ASE from data_model_2 where lable_ASE=0 and vectorfused is not null limit " + str(
        len(methods_padding)) + ";"
    result = cur.execute(sqli)
    for i in range(1, len(methods_padding)):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        lables.append(oneRow[1])
    return methods_padding, lables


def string_vector_to_int(methods_padding, oneRow):
    methods_padding = np.append(methods_padding, [list(map(int, oneRow[0][1:len(oneRow[0]) - 1].split(",")))], axis=0)
    return methods_padding


def float_revert_int(value_list):
    revert_value_list = []
    for i in value_list:
        revert_value_list.append(int(np.round(i)))
    value_list = revert_value_list
    return revert_value_list


def draw_confusion_matrix(y_true, y_pred, dic_lables):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    plt.cla()

    labels = []
    for key in dic_lables:
        labels.append(key)
    print_message(type(labels))
    # 小数转整数
    y_pred_int = float_revert_int(y_pred)
    sns.set()
    f, ax = plt.subplots()
    C2 = confusion_matrix(y_true, y_pred, labels=labels)
    print_message(C2)  # 打印出来看看
    sns.heatmap(C2, annot=True, fmt='.20g', ax=ax, cmap="YlGnBu")  # 画热力图
    ax.set_title('confusion matrix')  # 标题
    ax.set_xlabel('true')  # x轴
    ax.set_ylabel('predict')  # y轴
    save_pic(plt, '混淆矩阵')
    # plt.show()
    return plt


def save_pic(plt, file_name):
    # 创建目录
    import os, time
    dirs = 'C:\\Users\\chang\\Desktop\\日志工作空间\\实验图片\\'
    t = time.strftime('%Y-%m-%d-%H', time.localtime(int(time.time())))
    dirs = dirs + t
    file = dirs + '\\' + file_name + '.png'
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    if os.path.exists(file):
        os.remove(file)
    # 保存图片
    plt.savefig(file)  # 保存图片


def plot_value(y_true, y_pred):
    
    from sklearn.metrics import balanced_accuracy_score
    balance_accuracy = balanced_accuracy_score(y_true, y_pred)
    precision = sk.metrics.precision_score(y_true, y_pred)
    recall = sk.metrics.recall_score(y_true, y_pred)
    f1_value = sk.metrics.f1_score(y_true, y_pred)

    return balance_accuracy, precision, recall, f1_value


def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


def plot_graphs(history_loc, type_loc):
    import matplotlib.pyplot as plt
    plt.cla()

    plt.plot(history_loc.history[type_loc])
    plt.plot(history_loc.history['val_' + type_loc])
    plt.xlabel('Epochs')
    plt.ylabel(type_loc)
    plt.legend([type_loc, 'val_' + type_loc])
    save_pic(plt, type_loc)
    # plt.show()
    return plt


def str_revert_to_list(methods_padding, max_len):
    methods_padding_list = np.empty(shape=[0, max_len], dtype=int)

    for methods_padding_one in methods_padding:
        list1 = methods_padding_one.split(',')
        list1 = list(map(int, list1))
        methods_padding_list = np.append(methods_padding_list, [np.array(list1)], axis=0)  # 添加整行元素，axis=1添加整列元素
    return methods_padding_list


def get_object(save_absolute_path):
    summer_load = None
    with open(save_absolute_path, 'rb') as f:
        summer_load = pickle.load(f)  # read file and build object
    return summer_load


def save_object(summer_save, save_absolute_path):
    summer_save = pickle.dumps(summer_save)
    with open(save_absolute_path, 'wb') as f:  # open file with write-mode
        f.write(summer_save)  # serialize and save object


def get_parent(vec_chair):
    cur = conn.cursor()
    sqli = "select parentId from data_model_2 where seq=" + str(vec_chair[len(vec_chair) - 1]) + ";"
    try:
        result = cur.execute(sqli)
    except:
        print(sqli)
    oneRow = cur.fetchone()
    if oneRow[0] == 0:
        return vec_chair
    else:
        vec_chair.append(oneRow[0])
        return get_parent(vec_chair)


def get_chair_vec_int(vec_chair, i):
    vectorfused = None
    one_leaf_list = []
    vectorSemantic = None
    leaf = True
    message_type = i
    cur = conn.cursor()
    for one_leaf in vec_chair:
        sqli = "select REPLACE(REPLACE(a.vectorSemanticRawVec,'[',''),']','')," \
               "REPLACE(REPLACE(fusedMessageRawVec,'[',''),']','')," \
               "lable_ASE ,REPLACE(REPLACE(syntacticMessageRawVec,'[',''),']','') " \
               "from data_model_2 a  where seq=" + str(one_leaf) + ";"
        result = cur.execute(sqli)
        oneRow = cur.fetchone()
        vectorSemantic = oneRow[0]
        vectorfused = oneRow[1]
        lable_ASE = oneRow[2]
        syntacticMessage = oneRow[3]
        if vectorSemantic != "[]":
            if message_type == 1:
                if vectorfused is not None:
                    list_fused = vectorfused.split(',')
                    list_fused = list(map(float, list_fused))
                    one_leaf_list.extend(list_fused)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 2:
                if vectorSemantic is not None and len(vectorSemantic) > 0:
                    list_semantic = vectorSemantic.split(',')
                    list_semantic = list(map(float, list_semantic))
                    one_leaf_list.extend(list_semantic)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 3:
                if syntacticMessage is not None and len(syntacticMessage) > 0:
                    list_syntactic = syntacticMessage.split(',')
                    list_syntactic = list(map(float, list_syntactic))
                    one_leaf_list.extend(list_syntactic)
                    # one_leaf_list.extend([0,0,0,0])
    cur.close()
    return one_leaf_list


def get_chair_vec_raw(vec_chair, i):
    vectorfused = None
    one_leaf_list = []
    vectorSemantic = None
    leaf = True
    message_type = i
    cur = conn.cursor()
    for one_leaf in vec_chair:
        sqli = "select REPLACE(REPLACE(vectorSemanticRaw,'[',''),']','')," \
               "REPLACE(REPLACE(fusedMessageRaw,'[',''),']','')," \
               "lable_ASE ,REPLACE(REPLACE(syntacticMessageRaw,'[',''),']','') " \
               "from data_model_2 a  where seq=" + str(one_leaf) + ";"
        result = cur.execute(sqli)
        oneRow = cur.fetchone()
        vectorSemantic = oneRow[0]
        vectorfused = oneRow[1]
        syntacticMessage = oneRow[3]
        lable_ASE = oneRow[2]
        if vectorSemantic != "[]":
            if message_type == 1:
                if vectorfused is not None:
                    list_fused = vectorfused.split(',')
                    list_fused = list(map(str, list_fused))
                    one_leaf_list.extend(list_fused)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 2:
                if vectorSemantic is not None and len(vectorSemantic) > 0:
                    list_semantic = vectorSemantic.split(',')
                    list_semantic = list(map(str, list_semantic))
                    one_leaf_list.extend(list_semantic)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 3:
                if syntacticMessage is not None and len(syntacticMessage) > 0:
                    list_syntactic = syntacticMessage.split(',')
                    list_syntactic = list(map(str, list_syntactic))
                    one_leaf_list.extend(list_syntactic)
                    # one_leaf_list.extend([0,0,0,0])
    cur.close()
    return one_leaf_list


def print_message(message):
    t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(time.time())))
    print('时间：' + t, end=" ", flush=True)
    print(message, flush=True)


def repetition_list(a):
    repetition = 6
    c = []
    for a_one in a:
        b = []
        for a_single in a_one:
            for i in range(repetition):
                b.append(a_single)
        c.append(b)
    return c


def overlap_channel(a, b):
    element_num = 0
    height = 0
    width = 0
    if len(a.shape) == 2:
        element_num = a.shape[0]
        height = 1
        width = a.shape[1]
    elif len(a.shape) == 3:
        element_num = a.shape[0]
        height = a.shape[1]
        width = a.shape[2]
    else:
        raise Exception(print_message('通道叠加 不支持这类维度:' + str(a.shape)))

    a = a.reshape(element_num, 1, height, width)
    b = b.reshape(element_num, 1, height, width)
    a = a.transpose(1, 0, 2, 3)
    b = b.transpose(1, 0, 2, 3)
    c = np.vstack((a, b))
    c = c.transpose(1, 2, 3, 0)
    return c


def check_list_exist(A, B):
    exist = False
    for row in A:
        if (row == B).all():
            exist = True
    return exist


def get_convert_data(A, B):
    C = []
    for A_one in A:
        for B_one in B:
            if (np.array(A_one) == np.array(B_one)).all():
                C.append(A_one)
    return C


def cosDist(A, B):
    from scipy.spatial.distance import pdist
    return pdist(np.vstack([A, B]), 'cosine')  # np.sqrt(sum(np.power((A - B), 2)))


def get_normal_data(data_point, data_lable):
    neigh_num_a = 5
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
    return data_point_clear, data_lable_clear, abnormal_old


##仅支持0 - 1 分类
def change_abnormal(data_point, data_lable):
    neigh_num_a = 10
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
        else:
            ##修改noise标签
            data_point_clear.append(one_point)
            if one_lable == 0:
                data_lable_clear.append(1)
            else:
                data_lable_clear.append(0)
    return data_point_clear, data_lable_clear, abnormal_old


def clear_data(train_vec_tmp, train_labels, zero_index):
    group_num = 300
    train_vec_tmp_clear = []
    train_labels_clear = []
    abnormal_num = 0
    for b in range(int(len(train_vec_tmp) / group_num) + 1):
        begin = b * group_num + zero_index
        end = (b + 1) * group_num
        if end > len(train_vec_tmp):
            end = len(train_vec_tmp)
        train_vec_group_one, train_labels_group_one, abnormal_old = change_abnormal(train_vec_tmp[begin:end],
                                                                                    train_labels[begin:end])
        train_vec_tmp_clear.extend(train_vec_group_one)
        train_labels_clear.extend(train_labels_group_one)
        abnormal_num = abnormal_num + len(abnormal_old)
    print_message('noise数据量：' + str(abnormal_num))
    print_message('训练总数据量：' + str(len(train_vec_tmp)))
    train_vec_tmp = train_vec_tmp_clear
    train_labels = train_labels_clear
    return train_vec_tmp, train_labels


def play_title_message():
    from playsound import playsound
    playsound('D:/python/log_predict/file/结束提示.mp3')


def get_vector(leaf, lable):
    fused_leaf_vec = []
    semantic_leaf_vec = []
    syntatic_leaf_vec = []

    for one_leaf in leaf:
        vec_chain = []
        # 叶子节点放入 list 中
        vec_chain.append(one_leaf)
        for i in range(1, 4):
            # 根据list中的记录查询节点记录，组合向量
            one_leaf_vec = get_chair_vec_raw(vec_chain, i)
            if i == 1:
                fused_leaf_vec.append(one_leaf_vec)
            elif i == 2:
                semantic_leaf_vec.append(one_leaf_vec)
            else:
                syntatic_leaf_vec.append(one_leaf_vec)

    # 独热编码
    #     lable = tf.keras.utils.to_categorical(lable, num_classes=2)
    return fused_leaf_vec, semantic_leaf_vec, syntatic_leaf_vec, leaf, lable


def str_to_vec(semantic_tokenizer, semantic_train_vec, semantic_max_len):
    num_word = 50000  # 词典的词数
    voo_token = 'NONE'  # 指代缺失词

    trunc_type = 'post'
    semantic_method_sequence = semantic_tokenizer.texts_to_sequences(semantic_train_vec)
    semantic_method_padded = pad_sequences(semantic_method_sequence, maxlen=semantic_max_len,
                                           padding=trunc_type, truncating=trunc_type)
    return semantic_method_padded


def tokenizer_vec_use_tokenizer(tokenizer, train_vec_raw, verify_vec_raw, test_vec_raw):
    num_word = 500000
    voo_token = 'NONE'  # 指代缺失词
    train_vec_raw_convert = []
    verify_vec_raw_convert = []
    test_vec_raw_convert = []
    for i in train_vec_raw:
        train_vec_raw_convert.append(' '.join(i))

    for i in verify_vec_raw:
        verify_vec_raw_convert.append(' '.join(i))

    for i in test_vec_raw:
        test_vec_raw_convert.append(' '.join(i))
    train_vec_raw = tokenizer.texts_to_sequences(train_vec_raw_convert)
    verify_vec_raw = tokenizer.texts_to_sequences(verify_vec_raw_convert)
    test_vec_raw = tokenizer.texts_to_sequences(test_vec_raw_convert)
    return train_vec_raw, verify_vec_raw, test_vec_raw


def pad_vec(train_vec, verify_vec, test_vec, vec_length):
    trunc_type = 'post'
    train_vec = pad_sequences(train_vec,
                              maxlen=vec_length,
                              padding=trunc_type,
                              truncating=trunc_type)
    verify_vec = pad_sequences(verify_vec,
                               maxlen=vec_length,
                               padding=trunc_type,
                               truncating=trunc_type)
    test_vec = pad_sequences(test_vec,
                             maxlen=vec_length,
                             padding=trunc_type,
                             truncating=trunc_type)
    return train_vec, verify_vec, test_vec


def tokenizer_vec(vec_length, train_vec_raw, verify_vec_raw, test_vec_raw):
    num_word = 500000
    voo_token = 'NONE'  # 指代缺失词
    trunc_type = 'post'
    feature_all = []
    for i in train_vec_raw:
        feature_all.append(' '.join(i))
    for i in verify_vec_raw:
        feature_all.append(' '.join(i))
    for i in test_vec_raw:
        feature_all.append(' '.join(i))
    tokenizer = \
        Tokenizer(num_words=num_word, oov_token=voo_token)
    tokenizer.fit_on_texts(feature_all)
    train_vec_raw = tokenizer.texts_to_sequences(train_vec_raw)
    train_vec_raw = pad_sequences(train_vec_raw,
                                  maxlen=vec_length,
                                  padding=trunc_type,
                                  truncating=trunc_type)
    verify_vec_raw = tokenizer.texts_to_sequences(verify_vec_raw)
    verify_vec_raw = pad_sequences(verify_vec_raw,
                                   maxlen=vec_length,
                                   padding=trunc_type,
                                   truncating=trunc_type)
    test_vec_raw = tokenizer.texts_to_sequences(test_vec_raw)
    test_vec_raw = pad_sequences(test_vec_raw,
                                 maxlen=vec_length,
                                 padding=trunc_type,
                                 truncating=trunc_type)
    return train_vec_raw, verify_vec_raw, test_vec_raw


def get_tokenizer():
    cur = conn.cursor()
    sql = "select replace (vectorSemanticRaw,',',' '),logNum,seq ,replace (syntacticMessageRaw,',',' ') ,replace (fusedMessageRaw,',',' ') " \
          "from data_model_2 where " \
          "   fusedMessageRaw is not null ;"
    result = cur.execute(sql)
    semantic_methods = []
    syntatic_methods = []
    index = []

    fused_methods = []

    for i in range(result):
        oneRow = cur.fetchone()
        semantic_methods.append(oneRow[0])
        syntatic_methods.append(oneRow[3])
        fused_methods.append(oneRow[4])
    cur.close()
    # tokenize
    num_word = 50000  # 词典的词数
    voo_token = 'NONE'  # 指代缺失词
    trunc_type = 'post'
    semantic_tokenizer = Tokenizer(num_words=num_word, oov_token=voo_token)
    semantic_tokenizer.fit_on_texts(semantic_methods)

    # 处理语法信息
    syntatic_tokenizer = Tokenizer(num_words=num_word, oov_token=voo_token)
    syntatic_tokenizer.fit_on_texts(syntatic_methods)
    # 处理混合信息
    fused_tokenizer = Tokenizer(num_words=num_word, oov_token=voo_token)
    fused_tokenizer.fit_on_texts(fused_methods)
    return semantic_tokenizer, syntatic_tokenizer, fused_tokenizer


def get_index(sentence, word_index):
    sequence = []
    for word in sentence:
        try:
            sequence.append(word_index[word])
        except KeyError:
            pass
    return sequence


def word2vec(train_vec, verify_vec, test_vec, sentence_words_num):
    words_vec_num = 100
    vec_all = []
    for i in train_vec:
        vec_all.append([str(j) for j in i])
    for i in verify_vec:
        vec_all.append([str(j) for j in i])
    for i in test_vec:
        vec_all.append([str(j) for j in i])

    print_message(" 构建模型skip-gram")
    model = gensim.models.Word2Vec(vec_all, min_count=1, workers=8, sg=1, vector_size=words_vec_num)
    w2v_model = model

    for i in range(10):
        w2v_model.train(vec_all, total_examples=len(vec_all), epochs=1)

    ######
    train_vec = convert_vec_word2vec(train_vec, w2v_model, words_vec_num, sentence_words_num)
    verify_vec = convert_vec_word2vec(verify_vec, w2v_model, words_vec_num, sentence_words_num)
    test_vec = convert_vec_word2vec(test_vec, w2v_model, words_vec_num, sentence_words_num)
    #####

    return w2v_model, train_vec, verify_vec, test_vec


def convert_vec_word2vec(train_vec, w2v_model, words_vec_num, sentence_words_num):
    train_vec_new = []
    zero_list = []
    for i in range(words_vec_num):
        zero_list.append(0)
    for one_sentence in train_vec:
        one_sentence_vec = []
        for token in one_sentence:
            if len(one_sentence_vec) < sentence_words_num:
                embedding = w2v_model.wv[str(token)]
                one_sentence_vec.append(embedding)
        # padding词矩阵
        for i in range(sentence_words_num - len(one_sentence_vec)):
            one_sentence_vec.append(zero_list)
        train_vec_new.append(one_sentence_vec)
    return train_vec_new


def print_message(message):
    t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(time.time())))
    print('时间：' + t, end=" ", flush=True)
    print(message, flush=True)


def run_one_project(project_name, select_model):
    print(project_name, select_model)
    semantic_vec_length = 50
    syntatic_vec_length = 30
    fused_vec_length = 80
    seed = 1234  # random.randint(1, 1000)
    # 获取每个叶子节点
    print_message("获取节点")
    leaf = []
    lable = []

    # 获取数据集
    conn = get_connection()
    cur = conn.cursor()
    sqli = "select seq,CASE WHEN logNum > 0 THEN  1  ELSE 0 END " \
           "from  data_model_2 a where   " \
           "methodSeq in " \
           "(select seq from data_model_1 " \
           "where projectName in(" + project_name + ")) "

    result = cur.execute(sqli)
    for i in range(result):
        oneRow = cur.fetchone()
        leaf.append(oneRow[0])
        lable.append(oneRow[1])
    cur.close()
    # 打乱数据
    print_message('打乱数据')
    leaf, lable = shuffle_data(leaf, lable, seed)
    # 按比例切分数据集
    print_message('按比例切分数据集')
    leaf_train, lable_train, leaf_test, lable_test = stratified_shuffle_split(
        leaf, lable, 0.4, seed)
    leaf_test, lable_test, leaf_verify, lable_verify = stratified_shuffle_split(
        leaf_test, lable_test, 0.5, seed)

    print_message('获取特征')
    # 获取训练集合的特征
    fused_train_vec, \
    semantic_train_vec, \
    syntatic_train_vec, \
    train_leaf, \
    train_labels = get_vector(leaf_train, lable_train)
    # 获取验证集的特征
    fused_verify_vec, \
    semantic_verify_vec, \
    syntatic_verify_vec, \
    verify_leaf, \
    verify_labels = get_vector(leaf_verify, lable_verify)
    # 获取测试集的特征
    fused_test_vec, \
    semantic_test_vec, \
    syntatic_test_vec, \
    test_leaf, \
    test_labels = get_vector(leaf_test, lable_test)

    # tokenizer
    print_message('tokenizer')
    semantic_tokenizer, syntatic_tokenizer, fused_tokenizer = get_tokenizer()
    semantic_train_vec, semantic_verify_vec, semantic_test_vec \
        = tokenizer_vec_use_tokenizer(semantic_tokenizer,
                                      semantic_train_vec, semantic_verify_vec, semantic_test_vec)
    syntatic_train_vec, syntatic_verify_vec, syntatic_test_vec \
        = tokenizer_vec_use_tokenizer(syntatic_tokenizer,
                                      syntatic_train_vec, syntatic_verify_vec, syntatic_test_vec)
    fused_train_vec, fused_verify_vec, fused_test_vec \
        = tokenizer_vec_use_tokenizer(fused_tokenizer,
                                      fused_train_vec, fused_verify_vec, fused_test_vec)
    conn.close()
    print_message('word2vec')
    word2vec_semantic, semantic_train_vec, semantic_verify_vec, semantic_test_vec = \
        word2vec(semantic_train_vec, semantic_verify_vec, semantic_test_vec, semantic_vec_length)
    word2vec_syntatic, syntatic_train_vec, syntatic_verify_vec, syntatic_test_vec = \
        word2vec(syntatic_train_vec, syntatic_verify_vec, syntatic_test_vec, syntatic_vec_length)
    word2vec_fused, fused_train_vec, fused_verify_vec, fused_test_vec = \
        word2vec(fused_train_vec, fused_verify_vec, fused_test_vec, fused_vec_length)

    # SMOTE上采样
#     print_message('SMOTE上采样')

#     def combine_dim(list_1):
#         list_2 = list_1.reshape(list_1.shape[0], list_1.shape[1] * list_1.shape[2])
#         return list_2
#     def splite_dim(list_2, y, z):
#         return list_2.reshape(list_2.shape[0], y, z)
#     from imblearn.over_sampling import SMOTE

#     su = SMOTE(random_state=42)


#     if select_model == 1:
#         fused_train_vec = np.array(fused_train_vec)
#         y = fused_train_vec.shape[1]
#         z = fused_train_vec.shape[2]
#         fused_train_vec = combine_dim(fused_train_vec)
#         fused_train_vec, train_labels = \
#             su.fit_resample(fused_train_vec, train_labels)
#         fused_train_vec = splite_dim(fused_train_vec, y, z)
#     elif select_model == 2:
#         semantic_train_vec = np.array(semantic_train_vec)
#         y = semantic_train_vec.shape[1]
#         z = semantic_train_vec.shape[2]
#         semantic_train_vec = combine_dim(semantic_train_vec)
#         semantic_train_vec, train_labels = \
#             su.fit_resample(semantic_train_vec, train_labels)
#         semantic_train_vec = splite_dim(semantic_train_vec, y, z)
#     elif select_model == 4:
#         syntatic_train_vec = np.array(syntatic_train_vec)
#         y = syntatic_train_vec.shape[1]
#         z = syntatic_train_vec.shape[2]
#         syntatic_train_vec = combine_dim(syntatic_train_vec)
#         syntatic_train_vec, train_labels = \
#             su.fit_resample(syntatic_train_vec, train_labels)
#         syntatic_train_vec=splite_dim(syntatic_train_vec,y,z)
    from keras.layers import Convolution1D, MaxPooling1D

    print_message("组建模型")
    inp = None
    if select_model == 1:
        inp = Input(shape=np.array(fused_train_vec).shape[1:])
    elif select_model == 2:
        inp = Input(shape=np.array(semantic_train_vec).shape[1:])
    elif select_model == 4:
        inp = Input(shape=np.array(syntatic_train_vec).shape[1:])

    model = tf.keras.models.Sequential()
    model.add(inp)
#
    model.add(tf.keras.layers.LSTM(units=128, activation='tanh', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='sigmoid'))

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # 'categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
    model.summary()

    # 训练
    print_message("训练")
    history = []
    lr_reduce = keras.callbacks.ReduceLROnPlateau('val_loss', patience=3, factor=0.5, min_lr=0.000001)
    if select_model == 1:
        history = model.fit(np.array(fused_train_vec),
                            np.array(train_labels),
                            validation_data=(np.array(fused_verify_vec),
                                             np.array(verify_labels)),
                            batch_size=24,
                            callbacks=[lr_reduce],
                            epochs=num_epoch, verbose=2, shuffle=True)
    elif select_model == 2:
        history = model.fit(np.array(semantic_train_vec),
                            np.array(train_labels),
                            validation_data=(np.array(semantic_verify_vec),
                                             np.array(verify_labels)),
                            batch_size=24,
                            callbacks=[lr_reduce],
                            epochs=num_epoch, verbose=2, shuffle=True)

    elif select_model == 4:
        history = model.fit(np.array(syntatic_train_vec), np.array(train_labels),
                            validation_data=(np.array(syntatic_verify_vec),
                                             np.array(verify_labels)),
                            batch_size=24,
                            callbacks=[lr_reduce],
                            epochs=num_epoch, verbose=2, shuffle=True)

    # 单输出
    # 输出结果
    print_message("开始输出结果")
    predict_number = []
    if select_model == 1:
        predict_one_hot = model.predict(np.array(fused_test_vec))
    elif select_model == 2:
        predict_one_hot = model.predict(np.array(semantic_test_vec))

    elif select_model == 4:
        predict_one_hot = model.predict(np.array(syntatic_test_vec))

    # predict_number = [np.argmax(one_hot) for one_hot in predict_one_hot]
    # verify_number = [np.argmax(one_hot) for one_hot in test_labels]
    threshold = 0.5
    predict_one_hot = np.int64(predict_one_hot >= threshold)

    predict_number = predict_one_hot
    verify_number = test_labels

    y_true = verify_number
    y_pred = predict_number

    balance_accuracy_ase_all, precision_ase_all, recall_ase_all, f1_value_ase_all = plot_value(y_true, y_pred)
    feature_type = ""
    # 1-fused 2-语义
    # 4-语法（新）
    if select_model == 1:
        feature_type = "fused"
    elif select_model == 2:
        feature_type = "semantic"
    elif select_model == 4:
        feature_type = "syntatic"
    # plot_graphs(history, 'accuracy')
    # plot_graphs(history, 'loss')
    print_message("结束")
    save_performance('ASE'+feature_type, balance_accuracy_ase_all, precision_ase_all, recall_ase_all, f1_value_ase_all)

if __name__ == '__main__':

    project_name = "'"+one_project+"'"
#     project_name = "'zookeeper-master'"
    # 1-fused 2-语义
    # 4-语法（新）
    model_type = [1]#,2,4

    for select_model in model_type:
        run_one_project(project_name, select_model)


'zeppelin-master' 1
时间：2024-08-15 23:13:14 获取节点
时间：2024-08-15 23:13:16 打乱数据
时间：2024-08-15 23:13:16 按比例切分数据集
时间：2024-08-15 23:13:16 获取特征
时间：2024-08-15 23:14:40 tokenizer
时间：2024-08-15 23:15:44 word2vec
时间：2024-08-15 23:15:45  构建模型skip-gram
时间：2024-08-15 23:16:48  构建模型skip-gram
时间：2024-08-15 23:17:00  构建模型skip-gram
时间：2024-08-15 23:18:24 组建模型
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 80, 128)           117248    
                                                                 
 dropout (Dropout)           (None, 80, 128)           0         
                                                                 
 flatten (Flatten)           (None, 10240)             0         
                                                                 
 dense (Dense)               (None, 1)                 10241     
                                           

Epoch 49/100
1446/1446 - 100s - loss: 0.0657 - accuracy: 0.9689 - val_loss: 0.1568 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 100s/epoch - 69ms/step
Epoch 50/100
1446/1446 - 100s - loss: 0.0661 - accuracy: 0.9689 - val_loss: 0.1569 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 100s/epoch - 69ms/step
Epoch 51/100
1446/1446 - 107s - loss: 0.0659 - accuracy: 0.9692 - val_loss: 0.1570 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 107s/epoch - 74ms/step
Epoch 52/100
1446/1446 - 96s - loss: 0.0657 - accuracy: 0.9690 - val_loss: 0.1570 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 96s/epoch - 67ms/step
Epoch 53/100
1446/1446 - 96s - loss: 0.0659 - accuracy: 0.9692 - val_loss: 0.1571 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 96s/epoch - 66ms/step
Epoch 54/100
1446/1446 - 96s - loss: 0.0658 - accuracy: 0.9690 - val_loss: 0.1571 - val_accuracy: 0.9469 - lr: 1.0000e-06 - 96s/epoch - 67ms/step
Epoch 55/100
1446/1446 - 96s - loss: 0.0659 - accuracy: 0.9691 - val_loss: 0.1571 - val_accuracy: 0.9467 - lr: 1.0000e

In [2]:
num_epoch = 100  # 训练周期
feature_length = "10"
one_project = 'zeppelin-master'